In [ ]:
!pip install sacrebleu

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import sacrebleu

In [3]:
df = pd.read_csv("/kaggle/input/mansi-russian-parralel-corpus/overall_80K.csv")

In [4]:
df

,Unnamed: 0,target,source
0,0,Та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,Те мальчики не выполнят задание в назначенный ...
1,1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала."
2,2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах
3,3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит."
4,4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёр...",Мы никогда не забудем этого честного человека.
...,...,...,...
81141,2345,А̄нумн ка̄салахты аквтуп тамле о̄лнэ накыт ма̄...,"Мне кажется, что подобные случаи могут вызыват..."
81142,2346,А̄танэ нё̄тнэ̄г юил акван-атманэ.,Волосы аккуратно собраны сзади.
81143,2347,"Тох тай, культура сака тэ̄пгалан мед а̄тим.","В общем, культуры интенсивного потребления мед..."
81144,2348,"Тувыл Уэйтс ты музыкантыг ёт, Чарли Рич ос Фрэ...",Затем Уэйтс отправился на гастроли с такими му...


In [5]:
# чистим дф от мусора
df = df.drop(columns=[df.columns[0]])

In [6]:
df

,target,source
0,Та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,Те мальчики не выполнят задание в назначенный ...
1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала."
2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах
3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит."
4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёр...",Мы никогда не забудем этого честного человека.
...,...,...
81141,А̄нумн ка̄салахты аквтуп тамле о̄лнэ накыт ма̄...,"Мне кажется, что подобные случаи могут вызыват..."
81142,А̄танэ нё̄тнэ̄г юил акван-атманэ.,Волосы аккуратно собраны сзади.
81143,"Тох тай, культура сака тэ̄пгалан мед а̄тим.","В общем, культуры интенсивного потребления мед..."
81144,"Тувыл Уэйтс ты музыкантыг ёт, Чарли Рич ос Фрэ...",Затем Уэйтс отправился на гастроли с такими му...


In [7]:
train_df, test_val_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_val_df, test_size=0.5, random_state=42)

In [8]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [9]:
#model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
#tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

model = M2M100ForConditionalGeneration.from_pretrained("/kaggle/input/m2m-checkpoint")
tokenizer = M2M100Tokenizer.from_pretrained("/kaggle/input/m2m-checkpoint")

In [10]:
tokenizer.add_special_tokens({'additional_special_tokens': ['<mns_MNS>']})
model.resize_token_embeddings(len(tokenizer))

M2M100ScaledWordEmbedding(128105, 1024, padding_idx=1)

In [11]:
def preprocess_function(examples):
    inputs = tokenizer(examples['source'], truncation=True, padding='max_length', max_length=128)
    targets = tokenizer(examples['target'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = targets['input_ids']
    return inputs

In [12]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/64916 [00:00<?, ? examples/s]

Map:   0%|          | 0/8115 [00:00<?, ? examples/s]

Map:   0%|          | 0/8115 [00:00<?, ? examples/s]

In [13]:
train_dataset = train_dataset.remove_columns(['source', 'target', '__index_level_0__'])
val_dataset = val_dataset.remove_columns(['source', 'target', '__index_level_0__'])
test_dataset = test_dataset.remove_columns(['source', 'target', '__index_level_0__'])

In [14]:
import torch
torch.cuda.empty_cache()

In [15]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./mbart_m2m_finetuned",
    evaluation_strategy="no", 
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4, 
    weight_decay=0.01,
    save_total_limit=5, 
    save_strategy="epoch", 
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    report_to="none", 
    fp16=True, 
    load_best_model_at_end=False,  
)

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    bleu = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels])
    chrf = sacrebleu.corpus_chrf(decoded_preds, [decoded_labels])

    return {"bleu": bleu.score, "chrf": chrf.score}

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
500,2.135400
1000,0.515700
1500,0.444000
2000,0.416800
2500,0.382700
3000,0.369400
3500,0.359900
4000,0.353200
4500,0.320100
5000,0.315600


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instea

TrainOutput(global_step=12171, training_loss=0.3933286246096879, metrics={'train_runtime': 15725.5058, 'train_samples_per_second': 12.384, 'train_steps_per_second': 0.774, 'total_flos': 5.27516602246103e+16, 'train_loss': 0.3933286246096879, 'epoch': 2.999815145726785})

In [16]:
test_results = trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")
len(test_results.keys())

7

In [17]:
test_results.keys()

dict_keys(['test_loss', 'test_model_preparation_time', 'test_bleu', 'test_chrf', 'test_runtime', 'test_samples_per_second', 'test_steps_per_second'])

In [18]:
print(f"Тестовый BLEU: {test_results['test_bleu']}, ChrF: {test_results['test_chrf']}")

Тестовый BLEU: 44.918394768056196, ChrF: 55.83117881683063


In [63]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, MBartForConditionalGeneration, MBart50Tokenizer
import sacrebleu
import pandas as pd
import random
from tqdm import tqdm

model_checkpoint = "/kaggle/input/m2m-checkpoint"
model = M2M100ForConditionalGeneration.from_pretrained(model_checkpoint)
tokenizer = M2M100Tokenizer.from_pretrained(model_checkpoint)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

random.seed(42) 
# random_indices = random.sample(range(len(test_dataset)), 1000)
# random_test_samples = [test_dataset[i] for i in random_indices]
random_test_samples = test_dataset

def generate_translation(sample):
    input_ids = torch.tensor(sample['input_ids']).unsqueeze(0).to(device)  # Конвертируем список в тензор
    with torch.no_grad():
        generated_ids = model.generate(input_ids, max_length=200, num_beams=5, early_stopping=True)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

original_texts = []
correct_translations = []
model_translations = []

for sample in tqdm(random_test_samples):
    input_text = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
    correct_translation = tokenizer.decode(sample['labels'], skip_special_tokens=True)
    model_translation = generate_translation(sample)

    original_texts.append(input_text)
    correct_translations.append(correct_translation)
    model_translations.append(model_translation)
    
df_results = pd.DataFrame({
    "Original Text": original_texts,
    "Correct Translation": correct_translations,
    "Model Translation": model_translations
})

df_results.loc[:, 'Original Text'] = df_results['Original Text'].str.replace('__en__', '')
df_results.loc[:, 'Correct Translation'] = df_results['Correct Translation'].str.replace('__en__', '')
df_results.loc[:, 'Model Translation'] = df_results['Model Translation'].str.replace('__en__', '')
    
bleu_score = sacrebleu.corpus_bleu(df_results['Model Translation'].tolist(), 
                                   [df_results['Correct Translation'].tolist()]).score
chrf_score = sacrebleu.corpus_chrf(df_results['Model Translation'].tolist(), 
                                   [df_results['Correct Translation'].tolist()]).score

print(f"BLEU Score: {bleu_score}")
print(f"ChrF Score: {chrf_score}")

df_results.head(50)


100%|██████████| 8115/8115 [1:12:39<00:00,  1.86it/s]


BLEU Score: 21.664822458079087
ChrF Score: 51.97204285133187


,Original Text,Correct Translation,Model Translation
0,Пётр всего полведра воды принёс.,Петыр вēтра суп вит туп та тотас.,Пēтыр пуссын во̄т па̄л вит тотыс.
1,Он как я ростом стал.,Тав ам янытумый ēмтыс,Тав ам хумле яныгмаыг ēмтыс.
2,"Стоят лодки полные рыбы, для заготовки запасо...","Хл ха̄пыт нлэ̄гыт, та̄юм ва̄рнэ та̄юм савакыт...","Хл та̄глы ха̄пыт нлэ̄гыт, тэ̄нутыт щё̄питан м..."
3,Глаза имеющий человек его может увидеть.,Сам ōньщнэ хōтпан тав кāсалалаве.,Сама хум таве ка̄салакве вēрми.
4,"А я говорю: "" Наплевать, что больно, Маруся.","А а̄м ла̄вегум: "" Са̄лямак, ма̄ныр а̄ймы, Мар...","Ам ла̄ве̄гум: « Маруся, маныр а̄гмы."
5,"Пришли к купцу, домой зашли.","Щёлы хум палт ёхтысый., юв щялтсы̄й.","Ва̄тахум палт ёхтысы̄г, юв щалтсы̄г."
6,"Ты говоришь, как взрослые!",На яныгма̄хум хольт потыртэ̄гын!,"На ла̄вēгын, яныгхо̄тпат хольт!"
7,Он мы манси и ханты работающие в газете люди ...,Тав ма̄н ма̄ньщи ос ханты газетагувт рпитан м...,Тав ма̄н ма̄ньщи ос ханты газетат рпитан ма̄х...
8,"Затем человек, одетый в светлый халат, в свет...","Тувыл ань войкан сахи, войкан ня̄ра масум хум...","Тувыл посы супыл масхатым, посы ла̄гыл масхат..."
9,Ползающие звери - семь юров пусть ползают.,Ва̄кынэ й са̄т ю̄р вос ва̄кēгыт.,Во̄р й са̄т ю̄р вос ргалаве.


In [23]:
import shutil
shutil.make_archive("mbart_m2m_finetuned", 'zip', "/kaggle/working/mbart_m2m_finetuned/checkpoint-12171")

'/kaggle/working/mbart_m2m_finetuned.zip'

In [27]:
from IPython.display import FileLink

FileLink('mbart_m2m_finetuned.zip')

/kaggle/working/mbart_m2m_finetuned.zip